In [16]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import sklearn
import os
import PyPDF2

In [23]:
directory = "articles et bouquins"
corpus = []

for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text()
            with open("corpus.txt", "a", encoding="utf-8") as file2:
                file2.write(f"=== Contenu du fichier : {filename} ===\n")
                file2.write(text)
                file2.write("\n\n")

In [30]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

# Ensure you have the necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Read the content of the corpus.txt file
with open("corpus.txt", "r", encoding="utf-8") as file:
    content = file.read()

# Function to clean and preprocess text
def preprocess_text(text):
    text = re.sub(r' +', ' ', content)

    # Enlever les retours à la ligne successifs
    text = re.sub(r'\n+', '\n', content)

    # Supprimer les tirets au début de chaque phrase
    text = re.sub(r'^-', '', content, flags=re.MULTILINE)
    
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english') + stopwords.words('french'))
    words = [word for word in words if word not in stop_words]
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return ' '.join(words)

# Preprocess the content
cleaned_content = preprocess_text(content)

# Save the cleaned content back to the file
with open("corpus_cleaned.txt", "w", encoding="utf-8") as file:
    file.write(cleaned_content)

print("Text preprocessing complete. Cleaned content saved to corpus_cleaned.txt.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Text preprocessing complete. Cleaned content saved to corpus_cleaned.txt.
